# Project 1

## 1. Heuristic algorithms

In this exercise, we will implement 4 heuristic algorithms for link prediction of nodes in graph, i.e.

* Common neighbours (CN)
* Adamic Adar (AA)
* Resource Allocation (RA)
* Personalized PageRank (PPR)

We will look into details of these algorithms about how they process link prediction and how well they can achieve it.



### 1) Import library

Firstly, import the necessary libraries supporting all the exercises:

In [22]:
import sys
sys.path.insert(0, '..')

from tqdm import tqdm
import numpy as np
import torch
from torch_geometric.loader import DataLoader
from torch_geometric.datasets import Planetoid
from torch_geometric.transforms import RandomLinkSplit
import scipy.sparse as ssp


from typing import Tuple, List, Dict, Any, Optional
from torch import FloatTensor
from scipy.sparse import csr_matrix
from torch_geometric.utils import from_scipy_sparse_matrix

### 2) Load data and split 

In [23]:
dataset_name = 'Cora'  
path = '.'
undirected = True
val_pct = 0.2
test_pct = 0.1 
include_negatives = True
dataset = Planetoid(path, dataset_name)
transform = RandomLinkSplit(is_undirected=undirected,
                            num_val=val_pct,
                            num_test=test_pct,
                            add_negative_train_samples=include_negatives)
train_data, val_data, test_data = transform(dataset._data)

### 3) Get positive and negative edges

In the input data, there are positive and negative edges (as opposed to message passing edges). The following functions are for extracting them by RandomLinkSplit, and load the samples which will be actually input to algorithms by percentage or sample number.

In [24]:
def get_pos_neg_edges(data, sample_frac=1):
    """
    :param data: A train, val or test split returned by RandomLinkSplit
    :return: positive edge_index, negative edge_index.
    """
    device = data.edge_index.device
    edge_index = data['edge_label_index'].to(device)
    labels = data['edge_label'].to(device)
    pos_edges = edge_index[:, labels == 1].t()
    neg_edges = edge_index[:, labels == 0].t()
    if sample_frac != 1:
        n_pos = pos_edges.shape[0]
        np.random.seed(123)
        perm = np.random.permutation(n_pos)
        perm = perm[:int(sample_frac * n_pos)]
        pos_edges = pos_edges[perm, :]
        neg_edges = neg_edges[perm, :]
    return pos_edges.to(device), neg_edges.to(device)

pos_train_edge, neg_train_edge = get_pos_neg_edges(train_data)
pos_val_edge, neg_val_edge = get_pos_neg_edges(val_data)
pos_test_edge, neg_test_edge = get_pos_neg_edges(test_data)

train_data.pos_edges, train_data.neg_edges = pos_train_edge, neg_train_edge
val_data.pos_edges, val_data.neg_edges = pos_val_edge, neg_val_edge
test_data.pos_edges, test_data.neg_edges = pos_test_edge, neg_test_edge

### 4) Prepare adjacency matrix A

The adjacency matrix, denoted as A, represents a graph's connections. In a binary form, $A[i][j]$ is 1 if nodes i and j are connected and 0 otherwise. It succinctly captures the graph's structure.

Create the adjacency matrix A for further testing of heuristic algorithms with loaded data.

In [25]:
num_nodes = dataset._data.num_nodes
labels = torch.tensor([1] * pos_train_edge.size(0) + [0] * neg_train_edge.size(0))
edge_weight = torch.ones(dataset._data.edge_index.shape[1])
A = ssp.csr_matrix((edge_weight, (dataset._data.edge_index[0,:], dataset._data.edge_index[1,:])),shape=(num_nodes, num_nodes), dtype=float)

### 5) Complete the algorithms

There comes to heuristic algorithms themselves. Please based on your knowledge and previous assignments, fill the TODO parts in functions to complete the algorithms.

**Hint: refer to formulas of corresponding algorithms.**

In [26]:
def CN(A: csr_matrix, edge_index: torch.Tensor, batch_size: int = 100000) -> Tuple[FloatTensor, torch.Tensor]:
    """
    Common neighbours
    :param A: scipy sparse adjacency matrix
    :param edge_index: pyg edge_index (torch.Tensor of shape [2, num_edges])
    :param batch_size: int
    :return: Tuple containing a FloatTensor of scores and the pyg edge_index
    """
    edge_index = edge_index.t()
    link_loader = DataLoader(range(edge_index.size(0)), batch_size)
    scores = []
    for ind in tqdm(link_loader):
        src, dst = edge_index[ind, 0], edge_index[ind, 1]
        cur_scores = np.array(np.sum(A[src].multiply(A[dst]), 1)).flatten()
        scores.append(cur_scores)
    scores = np.concatenate(scores, 0)
    print(f'evaluated Common Neighbours for {len(scores)} edges')
    return torch.FloatTensor(scores), edge_index


def AA(A: csr_matrix, edge_index: torch.Tensor, batch_size: int = 100000) -> Tuple[FloatTensor, torch.Tensor]:
    """
    Adamic Adar
    :param A: scipy sparse adjacency matrix
    :param edge_index: pyg edge_index (torch.Tensor of shape [2, num_edges])
    :param batch_size: int
    :return: Tuple containing a FloatTensor of scores and the pyg edge_index
    """
    edge_index = edge_index.t()
    multiplier = 1 / np.log(A.sum(axis=0))
    multiplier[np.isinf(multiplier)] = 0
    A_ = A.multiply(multiplier).tocsr()
    link_loader = DataLoader(range(edge_index.size(0)), batch_size)
    scores = []
    for ind in tqdm(link_loader):
        src, dst = edge_index[ind, 0], edge_index[ind, 1]
        cur_scores = np.array(np.sum(A[src].multiply(A_[dst]), 1)).flatten()
        scores.append(cur_scores)
    scores = np.concatenate(scores, 0)
    print(f'evaluated Adamic Adar for {len(scores)} edges')
    return torch.FloatTensor(scores), edge_index


def RA(A: csr_matrix, edge_index: torch.Tensor, batch_size: int = 100000) -> Tuple[FloatTensor, torch.Tensor]:
    """
    Resource Allocation https://arxiv.org/pdf/0901.0553.pdf
    :param A: scipy sparse adjacency matrix
    :param edge_index: pyg edge_index (torch.Tensor of shape [2, num_edges])
    :param batch_size: int
    :return: Tuple containing a FloatTensor of scores and the pyg edge_index
    """
    edge_index = edge_index.t()
    multiplier = 1 / A.sum(axis=0)
    multiplier[np.isinf(multiplier)] = 0
    A_ = A.multiply(multiplier).tocsr()
    link_loader = DataLoader(range(edge_index.size(0)), batch_size)
    scores = []
    for ind in tqdm(link_loader):
        src, dst = edge_index[ind, 0], edge_index[ind, 1]
        cur_scores = np.array(np.sum(A[src].multiply(A_[dst]), 1)).flatten()
        scores.append(cur_scores)
    scores = np.concatenate(scores, 0)
    print(f'evaluated Resource Allocation for {len(scores)} edges')
    return torch.FloatTensor(scores), edge_index


def PPR(A, edge_index):
    """
    The Personalized PageRank heuristic score.
    Need to install fast_pagerank by "pip install fast-pagerank"
    Too slow for large datasets now.
    :param A: A CSR matrix using the 'message passing' edges
    :param edge_index: The supervision edges to be scored
    :return:
    """
    edge_index = edge_index.T
    from fast_pagerank import pagerank_power
    num_nodes = A.shape[0]
    src_index, sort_indices = torch.sort(edge_index[:, 0])
    dst_index = edge_index[sort_indices, 1]
    edge_reindex = torch.stack([src_index, dst_index])
    scores = []
    visited = set([])
    j = 0
    for i in tqdm(range(edge_reindex.shape[1])):
        if i < j:
            continue
        src = edge_reindex[0, i]
        personalize = np.zeros(num_nodes)
        personalize[src] = 1
        # get the ppr for the current source node
        ppr = pagerank_power(A, p=0.85, personalize=personalize, tol=1e-7)
        j = i
        # get ppr for all links that start at this source to save recalculating the ppr score
        while edge_reindex[0, j] == src:
            j += 1
            if j == edge_reindex.shape[1]:
                break
        all_dst = edge_reindex[1, i:j]
        cur_scores = ppr[all_dst]
        if cur_scores.ndim == 0:
            cur_scores = np.expand_dims(cur_scores, 0)
        scores.append(np.array(cur_scores))

    scores = np.concatenate(scores, 0)
    print(f'evaluated PPR for {len(scores)} edges')
    return torch.FloatTensor(scores), edge_reindex

### 6) Test heuristic algorithms

In [27]:
print('starting training')
CN_scores, edge_index = CN(A, dataset._data.edge_index)
print(CN_scores)
PA_scores, edge_index = RA(A, dataset._data.edge_index)
print(PA_scores)
AA_scores, edge_index = AA(A, dataset._data.edge_index)
print(AA_scores)
PPR_scores, edge_index = PPR(A, dataset._data.edge_index)
print(PPR_scores)

starting training


100%|██████████| 1/1 [00:00<00:00, 69.37it/s]


evaluated Common Neighbours for 10556 edges
tensor([0., 1., 1.,  ..., 2., 3., 2.])


  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:00<00:00, 70.28it/s]
/scratch/ipykernel_3437702/2261758863.py:30: RuntimeWarning: divide by zero encountered in divide
  multiplier = 1 / np.log(A.sum(axis=0))


evaluated Resource Allocation for 10556 edges
tensor([0.0000, 0.3333, 0.2500,  ..., 0.5000, 0.5303, 0.5000])


100%|██████████| 1/1 [00:00<00:00, 78.75it/s]


evaluated Adamic Adar for 10556 edges
tensor([0.0000, 0.9102, 0.7213,  ..., 1.4427, 1.7287, 1.4427])


100%|██████████| 10556/10556 [00:12<00:00, 824.91it/s]

evaluated PPR for 10556 edges
tensor([0.1125, 0.0734, 0.0991,  ..., 0.1089, 0.0896, 0.0837])


## 2. Neural Network

Heuristic algorithms works in a intuitive way to capture the underneath patterns among nodes and links in graph. Now we try another thought with neural networks, which implement the same purpose as heuristic algorithm but through **learning** capacities of models. In a nutshell, we let the models themselves to find the pattern, instead of we giving primitive guess and designing. We will try two kinds of neural networks:

* MLP: Multi-layer Perceptron
* GCN: Graph convolutional network

Note that in this project, both models only possess basic structures, and training methods are also primitive without fine-tunning. Hence, this project might not be able to give outcomes as good as heuristic algorithms, but aims to help you get familiar with the idea of how to construct a basic neural network training.

### 1) Import torch

torch provides with abundant modularized functions and classes especially for neural networks. Import corresponding ones with:

In [28]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch_geometric.utils import to_scipy_sparse_matrix
from torch_geometric.nn import GCNConv

### 2) Load data

Load data and split it into training set, validation set and testing set.

In [29]:
dataset = Planetoid(root='.', name='Cora') 
data = dataset[0]
adj = to_scipy_sparse_matrix(data.edge_index)
print(f'Overview of dataset:')
print(f'Classes of labels: {set(list(i.item() for i in data.y))}')
print(f'row of adjacency matrix A: {adj.row}')
print(f'col of adjacency matrix A: {adj.col}')
print(f'data of adjacency matrix A: {adj.data}')

# split_data
transform = RandomLinkSplit(is_undirected=True,
                            num_val=0.2,
                            num_test=0.1,
                            add_negative_train_samples=True
                            )
train_data, val_data, test_data = transform(data)

Overview of dataset:
Classes of labels: {0, 1, 2, 3, 4, 5, 6}
row of adjacency matrix A: [ 633 1862 2582 ...  598 1473 2706]
col of adjacency matrix A: [   0    0    0 ... 2707 2707 2707]
data of adjacency matrix A: [1. 1. 1. ... 1. 1. 1.]


### 3) define model

#### 1. MLP: Multi-layer perceptron

MLP is universal approximator consisting of fully connected layers, which is capable of finding patterns of almost all datasets, but of course, in theory, and only to some degrees. Before a threshold, inceasing number of layers can indeed be more powerful to learn from data, but afterwards, it will cause plenty of problems. If it interests you, feel free to search some classic problems when training neural networks:

* training not converges
* overfitting vs underfitting
* gradient explosion / vanishing
* bias-variance tradeoff
* computational complexity of MLP

Here we only set up 3 layers for MLP.

In [30]:
class MLP(torch.nn.Module):
    """ Multiple Layers' Perceptron
    
    Params
    ----------
    nfeat : dimension of input 
    nhid : dim of hidden neurons
    nclass : number of classes ground truth
    dropout : dropout probability
    with_bias: None
    """
    def __init__(self, num_feat, nclass, n_hidden, dropout=0.5, with_bias=True):
        super(MLP, self).__init__()
        self.num_feat = num_feat
        self.class_label = nclass
        self.n_hidden = n_hidden
        self.linear1 = nn.Linear(self.num_feat, self.n_hidden)
        self.linear2 = nn.Linear(self.n_hidden, self.n_hidden)
        self.linear3 = nn.Linear(self.n_hidden, self.class_label)
        self.dropout = dropout
        
    def reset_parameters(self):
        """Initialize"""
        self.linear1.reset_parameters()
        self.linear2.reset_parameters()
        self.linear3.reset_parameters()

    def forward(self, data):
        """Forward Your code"""
        x = F.dropout(self.linear1(data.x), p=self.dropout, training=self.training)
        x = F.dropout(self.linear2(x), p=self.dropout, training=self.training)
        x = F.dropout(self.linear3(x), p=self.dropout, training=self.training)        
        return F.log_softmax(x, dim=1)

#### 2 Proposed SOTA: GCN: Graph convolutional network

Convolution neural network transfers the convolutional operation on singal processing to deep learning realm, which captures local features with convolutional kernels (i.e. filters) on different form of data (e.g. images, signals). It requires extremely less parameters than MLP but usually gives better performance, if the model fits the situation well.

Graph convolutional network (GCN) is especially designed for graph-structured data, which combine the feature of graph and convolutional neural network. Here we test a GCN model with 2 GCN layers.

In [31]:
class GCN(torch.nn.Module):
    """ Two Layers' GCN
    
    Params
    ----------
    nfeat : dimension of input 
    nhid : dim of hidden neurons
    nclass : number of classes ground truth
    dropout : dropout probability
    with_bias: None
    """

    def __init__(self, nfeat, nclass, nhid, dropout=0.5, with_bias=True):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(nfeat, nhid, bias=with_bias, activation=F.relu)
        self.conv2 = GCNConv(nhid, nclass, bias=with_bias)
        self.dropout = dropout
        
    def reset_parameters(self):
        """Initialize"""
        self.conv1.reset_parameters()
        self.conv2.reset_parameters()

    def forward(self, data, features=None):
        """Forward Your code"""
        x, edge_index = data.x, data.edge_index 
        x = F.relu(self.conv1(x, edge_index)) 
        x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

### 4) Train & Test

Then we define the process of training and testing.

Usually, training of a neural network includes:

* send train data into model and get output
* calculate loss between output and ground truth (i.e. target)
* conduct back-propagation to get gradients
* update parameters of model with gradients and optimizer
* repeat the whole process for multiple epochs

Since testing is to evaluate how well the trained model can already learn , we send test data into trained model on a new data (split from original dataset but not invovlved in training phase) and only once.

In [32]:
def train(model, data, lr=0.001, weight_decay=5e-4, epochs=200):
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    labels = data.y
    train_mask = data.train_mask
    best_loss_val = 100

    for i in range(epochs):
        model.train()
        optimizer.zero_grad()
        output = model(data)
        # only training nodes will be used to calculate loss 
        loss = F.nll_loss(output[train_mask], labels[train_mask]) 
        loss.backward()
        optimizer.step()

        if i % 10 == 0:
            print('Epoch {}, training loss: {}'.format(i, loss.item()))

@torch.no_grad()
def test(model, data):
    """Evaluate GAT performance on test set.

    """
    model.eval()
    test_mask = data.test_mask
    labels = data.y 
    output = model(data) 
    loss_test = F.nll_loss(output[test_mask], labels[test_mask])
    preds = output[test_mask].argmax(1) 
    acc_test = preds.eq(labels[test_mask]).cpu().numpy().mean() 
    print("Test set results:",
          "loss= {:.4f}".format(loss_test.item()),
          "accuracy= {:.4f}".format(acc_test))
    return preds, output, acc_test.item()

num_feat = data.num_features
n_hidden = 16
labels = data.y  
nclass = labels.max().item()+1
device = 'cpu' # device ='cuda'
train_data = train_data.to(device)
test_data = test_data.to(device)

### 5) Training

#### 1. Proposed Baseline MLP

In [33]:
model = MLP(num_feat, nclass, n_hidden, dropout=0.5, with_bias=True)
model = model.to(device)
train(model, train_data, epochs=100)
preds, output, acc_test = test(model, test_data)

Epoch 0, training loss: 1.9597594738006592
Epoch 10, training loss: 1.9177745580673218
Epoch 20, training loss: 1.853309154510498
Epoch 30, training loss: 1.7658307552337646
Epoch 40, training loss: 1.6853845119476318
Epoch 50, training loss: 1.6468523740768433
Epoch 60, training loss: 1.5682454109191895
Epoch 70, training loss: 1.5421398878097534
Epoch 80, training loss: 1.545398473739624
Epoch 90, training loss: 1.4439668655395508
Test set results: loss= 1.6905 accuracy= 0.5430
